In [2]:
import os, math
import pandas as pd
import numpy as np

In [3]:
from gensim.models import Word2Vec as w2v
from os import path as osp

root = osp.dirname(os.getcwd())
model = w2v.load(osp.join(root,'model/stackexchange_model'))


In [4]:
def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals

In [5]:
cvs = pd.read_csv('prc_data.csv', sep='\t')
cvs = cvs.set_index('Unnamed: 0')

prc_description = "ne datum analyst knowledge python R"

In [6]:
word_value = {}
similar_words_needed = 2
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
        print similar_words[i], word_value[similar_words[i]] 

ne 1
je 0.636240243912
ke 0.629526257515
datum 1
datums 0.602979719639
wgs 0.574095129967
analyst 1
analysts 0.489208996296
engineer 0.480672895908
knowledge 1
expertise 0.707411766052
knowlege 0.643639862537
python 1
perl 0.720492959023
ruby 0.67925286293
r 1
l 0.67971944809
w 0.638600409031


In [8]:
no_of_cv = 150

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        try:
            if word in cvs.loc(0)['skill'][i].split() or word in cvs.loc(0)['exp'][i].split():
                count[word] += 1
        except:
            pass
    if (count[word] == 0):
        count[word] = 1
    idf[word] = math.log(no_of_cv/count[word])
print count
print idf

{u'knowlege': 1, u'wgs': 1, 'knowledge': 35, u'ke': 1, 'python': 4, 'datum': 13, u'l': 1, u'datums': 1, 'ne': 4, 'r': 1, u'expertise': 9, u'analysts': 1, u'w': 1, u'je': 1, u'perl': 1, u'ruby': 3, 'analyst': 5, u'engineer': 20}
{u'knowlege': 5.0106352940962555, u'wgs': 5.0106352940962555, 'knowledge': 1.3862943611198906, u'ke': 5.0106352940962555, 'python': 3.6109179126442243, 'datum': 2.3978952727983707, u'l': 5.0106352940962555, u'datums': 5.0106352940962555, 'ne': 3.6109179126442243, 'r': 5.0106352940962555, u'expertise': 2.772588722239781, u'analysts': 5.0106352940962555, u'w': 5.0106352940962555, u'je': 5.0106352940962555, u'perl': 5.0106352940962555, u'ruby': 3.912023005428146, 'analyst': 3.4011973816621555, u'engineer': 1.9459101490553132}


In [9]:
score = {}
for i in range(no_of_cv):
    score[i] = 0
    try:
        for word in word_value.keys():
            tf = cvs.loc(0)['skill'][i].split().count(word) + cvs.loc(0)['exp'][i].split().count(word)
            score[i] += word_value[word]*tf*idf[word]
    except:
        pass

In [10]:
sorted_list = []
for i in range(no_of_cv):
    sorted_list.append((score[i], i))
    
sorted_list.sort(reverse \
                 = True)

for s, i in sorted_list:
    print list(cvs)[i], ':', s

cv174 : 20.1580663963
cv215 : 15.3508679102
cv159 : 13.4224679699
cv150 : 12.1986801511
cv107 : 11.0044445332
cv233 : 10.8327537379
cv211 : 8.60813018641
cv126 : 7.39510754656
cv197 : 7.22183582529
cv199 : 7.1936858184
cv110 : 6.80239476332
cv129 : 6.70080001369
cv141 : 6.38350663488
cv192 : 5.74555151845
cv135 : 5.61207759914
cv122 : 5.61207759914
cv109 : 5.61207759914
cv175 : 5.31450565257
cv123 : 5.29460342386
cv124 : 4.35925715733
cv142 : 4.33654364819
cv119 : 4.26858780585
cv149 : 4.15888308336
cv172 : 4.04354718741
cv164 : 3.78418963392
cv111 : 3.70793498876
cv217 : 3.61091791264
cv201 : 3.40119738166
cv16 : 3.34765624566
cv148 : 3.34765624566
cv102 : 3.25698689417
cv193 : 2.77258872224
cv177 : 2.77258872224
cv204 : 2.3978952728
cv139 : 2.3978952728
cv120 : 2.32164062764
cv1.txt : 2.32164062764
cv226 : 1.96136188454
cv207 : 1.96136188454
cv194 : 1.96136188454
cv105 : 1.96136188454
cv213 : 1.87069253305
cv187 : 1.87069253305
cv113 : 1.87069253305
cv232 : 1.38629436112
cv229 : 1.38